In [1]:
import torch
from torch import nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.autograd import Variable

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler


In [2]:
#Read in the dataframe from the CSV file
df = pd.read_csv('Strat_Temp_AOD_Flux.csv')

In [3]:
#Extract lat, lon, and dates
lon_vals = df['lon'].unique()
lat_vals = df['lat'].unique()
dates = df['date'].unique()

In [4]:
#Create 16x48x120 tensor for Temperature data
data_tot = torch.empty(size=(16,48,120))
for i in range(len(lat_vals)):
    for j in range(len(lon_vals)):
        temp = df[(df['lat'] == lat_vals[i]) & (df['lon'] == lon_vals[j])]
        data_tot[i,j,:] = torch.from_numpy(np.array(temp['T'])).float()
#data_tot contains all the temperatures from all spatial points (16x48) with 120 time steps

In [5]:
# This function creates sliding window and normalizes X-values (inputs to the CNN)
def sliding_window(dates, data, seq_len, out_len): 
    x, y, start_dates = [], [], []
    
    for j in range(len(data[0,0]) - seq_len - out_len): #create sliding windows
        _x = data[:,:,j:(j+seq_len)]
        _y = data[:,:, (j+seq_len):(j+seq_len+out_len)]
        x_date = dates[j]
        y_date = dates[j+seq_len]
        x.append(_x)
        y.append(_y)
        start_dates.append((x_date, y_date))
    #Normalize
    x = np.array(x)
    xmax = x.max()
    xmin = x.min()
    x_norm = (x - xmin) / (xmax - xmin)
    return start_dates, x_norm, np.array(y)

In [6]:
#Training set A contains first 38 time steps
#Training set B contains first 49 time steps
sequence_len = 24
output_len = 6
x_start_dates, x_tot, y_tot =  sliding_window(dates, np.array(data_tot), sequence_len, output_len)

In [7]:
#Grab training set A x-temperatures
x_train_A = torch.from_numpy(x_tot[:31,:,:,:])

In [8]:
#Grab training set A y-temperatures
y_train_A = torch.from_numpy(y_tot[:31,:,:,:])

In [9]:
#Grab training set B x-temperatures
x_train_B = torch.from_numpy(x_tot[:42,:,:,:])

In [10]:
#Grab training set B y-temperatures
y_train_B = torch.from_numpy(y_tot[:42,:,:,:])

In [11]:
#Grab test set A x-temperatures
x_test_A = torch.from_numpy(x_tot[31:,:,:,:])

In [12]:
#Grab test set A y-temperatures
y_test_A = torch.from_numpy(y_tot[31:,:,:,:])

In [13]:
#Grab test set B x-temperatures
x_test_B = torch.from_numpy(x_tot[42:,:,:,:])

In [14]:
#Grab test set B y-temperatures
y_test_B = torch.from_numpy(y_tot[42:,:,:,:])

In [15]:
#We must convert the dimensions to [1, number of time steps, 16,48] to input into the CNN
x_train_A = torch.permute(x_train_A, (0,3,1,2))
y_train_A = torch.permute(y_train_A, (0,3,1,2))

x_train_B = torch.permute(x_train_B, (0,3,1,2))
y_train_B = torch.permute(y_train_B, (0,3,1,2))

x_test_A = torch.permute(x_test_A, (0,3,1,2))
y_test_A = torch.permute(y_test_A, (0,3,1,2))

x_test_B = torch.permute(x_test_B, (0,3,1,2))
y_test_B = torch.permute(y_test_B, (0,3,1,2))

In [16]:
#This function converts all sets to list of 16x48xtime tensors!!
def list_of_tensors(tensor):
    list_tensor = []
    for i in range(len(tensor)):
        list_tensor.append(tensor[i])
    return list_tensor

In [17]:
#For the input to the CNN we must get a list of tensors size (1,time_steps,16,48)
x_train_A = list_of_tensors(x_train_A)
y_train_A = list_of_tensors(y_train_A)

x_train_B = list_of_tensors(x_train_B)
y_train_B = list_of_tensors(y_train_B)

x_test_A = list_of_tensors(x_test_A)
y_test_A = list_of_tensors(y_test_A)

x_test_B = list_of_tensors(x_test_B)
y_test_B = list_of_tensors(y_test_B)

In [18]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        #24 in-channels for context, 6 out-channels for horizon, kernel size 3x3
        self.conv1 = nn.Conv2d(24, 6, 3, padding='same') 
        #self.conv2 = nn.Conv2d(24, 6, 3, padding='same')
    
    def forward(self, x):
        x = self.conv1(x)
        return x

In [19]:
#The latitude weights are the cosines of the latitude values

def weighted_RMSE(output, target):
    lat_weights = [ 0.98611103,  0.85521133,  0.72431164,  0.59341195,  0.46251225,
        0.33161256,  0.20071286,  0.06981317, -0.06108652, -0.19198622,
       -0.32288591, -0.45378561, -0.5846853 , -0.71558499, -0.84648469,
       -0.97738438] #Latitudes converted to radians
    loss = torch.mean(torch.mul((output - target)**2, torch.tensor(np.cos(lat_weights))[:,None]))
    return torch.sqrt(loss)

In [20]:
#Training loop for training set A Temperature only
n_epochs = 8000 #Can tune this parameter
learning_rate = 1e-3 #Can tune this parameter

cnn = CNN()
#criterion = weighted_RMSE()
optimizer = optim.Adam(cnn.parameters(), lr = learning_rate)

for i in range(n_epochs):
    avg_loss = 0
    for j in range(len(x_train_A)):
        cnn.train()
        outputs = cnn(x_train_A[j])
        optimizer.zero_grad()
    
        loss = weighted_RMSE(outputs, y_train_A[j])
        avg_loss += loss
        loss.backward()
    
        optimizer.step()
    avg_loss = avg_loss/len(x_train_A)
    avg_loss_test = 0
    for k in range(len(x_test_A)):
        cnn.eval()
        valid = cnn(x_test_A[k])
        test_loss = weighted_RMSE(valid, y_test_A[k])
        avg_loss_test += test_loss
    avg_loss_test = avg_loss_test/len(x_test_A)
    if i %500 == 0 or i == n_epochs-1:
        print("Epoch: %d, loss: %1.5f test loss:  %1.5f " %(i, avg_loss , avg_loss_test))
        
#Use torch.save() to save your model so you don't have to train another

Epoch: 0, loss: 191.31708 test loss:  190.10532 
Epoch: 500, loss: 46.32251 test loss:  44.68679 
Epoch: 1000, loss: 41.69821 test loss:  41.33567 
Epoch: 1500, loss: 37.69879 test loss:  38.20395 
Epoch: 2000, loss: 33.94496 test loss:  35.00249 
Epoch: 2500, loss: 30.30500 test loss:  31.68357 
Epoch: 3000, loss: 26.73129 test loss:  28.26242 
Epoch: 3500, loss: 23.20609 test loss:  24.76437 
Epoch: 4000, loss: 19.72366 test loss:  21.21294 
Epoch: 4500, loss: 16.28521 test loss:  17.62933 
Epoch: 5000, loss: 12.89880 test loss:  14.03519 
Epoch: 5500, loss: 9.58604 test loss:  10.46069 
Epoch: 6000, loss: 6.40503 test loss:  6.96743 
Epoch: 6500, loss: 3.56732 test loss:  3.76964 
Epoch: 7000, loss: 1.94489 test loss:  1.89876 
Epoch: 7500, loss: 1.68755 test loss:  1.71597 
Epoch: 7999, loss: 1.63285 test loss:  1.71460 


In [21]:
#Training loop for training set B Temperature only
n_epochs = 8000 #Can tune this parameter
learning_rate = 1e-3 #Can tune this parameter

cnn_B = CNN()
#criterion = RMSELoss()
optimizer = optim.Adam(cnn_B.parameters(), lr = learning_rate)

for i in range(n_epochs):
    avg_loss = 0
    for j in range(len(x_train_B)):
        cnn_B.train()
        outputs = cnn_B(x_train_B[j])
        optimizer.zero_grad()
    
        loss = weighted_RMSE(outputs, y_train_B[j])
        avg_loss += loss
        loss.backward()
    
        optimizer.step()
    avg_loss = avg_loss/len(x_train_B)
    avg_loss_test = 0
    for k in range(len(x_test_B)):
        cnn_B.eval()
        valid = cnn_B(x_test_B[k])
        test_loss = weighted_RMSE(valid, y_test_B[k])
        avg_loss_test += test_loss
    avg_loss_test = avg_loss_test/len(x_test_B)
    if i %500 == 0 or i == n_epochs-1:
        print("Epoch: %d, loss: %1.5f test loss:  %1.5f " %(i, avg_loss , avg_loss_test))
        
#Use torch.save() to save your model so you don't have to train another

Epoch: 0, loss: 190.96670 test loss:  189.32624 
Epoch: 500, loss: 43.64102 test loss:  44.00983 
Epoch: 1000, loss: 38.04231 test loss:  39.98075 
Epoch: 1500, loss: 33.04848 test loss:  35.83627 
Epoch: 2000, loss: 28.27120 test loss:  31.40964 
Epoch: 2500, loss: 23.61926 test loss:  26.77165 
Epoch: 3000, loss: 19.07447 test loss:  22.00208 
Epoch: 3500, loss: 14.64767 test loss:  17.17141 
Epoch: 4000, loss: 10.38297 test loss:  12.36132 
Epoch: 4500, loss: 6.40702 test loss:  7.72254 
Epoch: 5000, loss: 3.16916 test loss:  3.72569 
Epoch: 5500, loss: 1.84230 test loss:  1.82869 
Epoch: 6000, loss: 1.68984 test loss:  1.63855 
Epoch: 6500, loss: 1.65377 test loss:  1.63728 
Epoch: 7000, loss: 1.63191 test loss:  1.64318 
Epoch: 7500, loss: 1.61642 test loss:  1.65060 
Epoch: 7999, loss: 1.60510 test loss:  1.65883 
